This notebook will be used to combine datasets that will be used for modelling 

1. Combine Train and Weather
2. Combine Test and Weather

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':20})

from haversine import haversine
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
train_df = pd.read_csv('../assets/train.csv')
weather_wkly = pd.read_csv('../assets/weather_wkly_clean_1.csv')
test_df = pd.read_csv('../assets/test.csv')
train_df.columns = train_df.columns.str.lower()
test_df.columns = test_df.columns.str.lower()

# Clean Train

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    10506 non-null  object 
 1   address                 10506 non-null  object 
 2   species                 10506 non-null  object 
 3   block                   10506 non-null  int64  
 4   street                  10506 non-null  object 
 5   trap                    10506 non-null  object 
 6   addressnumberandstreet  10506 non-null  object 
 7   latitude                10506 non-null  float64
 8   longitude               10506 non-null  float64
 9   addressaccuracy         10506 non-null  int64  
 10  nummosquitos            10506 non-null  int64  
 11  wnvpresent              10506 non-null  int64  
dtypes: float64(2), int64(4), object(6)
memory usage: 985.1+ KB


In [4]:
#convert to datetime format and create new columns for year,month and week
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['week'] = train_df['date'].dt.isocalendar().week

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    10506 non-null  datetime64[ns]
 1   address                 10506 non-null  object        
 2   species                 10506 non-null  object        
 3   block                   10506 non-null  int64         
 4   street                  10506 non-null  object        
 5   trap                    10506 non-null  object        
 6   addressnumberandstreet  10506 non-null  object        
 7   latitude                10506 non-null  float64       
 8   longitude               10506 non-null  float64       
 9   addressaccuracy         10506 non-null  int64         
 10  nummosquitos            10506 non-null  int64         
 11  wnvpresent              10506 non-null  int64         
 12  year                    10506 non-null  int64 

In [6]:
train_df.drop(columns=['date','address', 'block','street','trap','addressnumberandstreet','addressaccuracy'],inplace=True)
#drop this columns as it is not useful for the model

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   species       10506 non-null  object 
 1   latitude      10506 non-null  float64
 2   longitude     10506 non-null  float64
 3   nummosquitos  10506 non-null  int64  
 4   wnvpresent    10506 non-null  int64  
 5   year          10506 non-null  int64  
 6   month         10506 non-null  int64  
 7   week          10506 non-null  UInt32 
dtypes: UInt32(1), float64(2), int64(4), object(1)
memory usage: 626.0+ KB


In [8]:
train_df= train_df.groupby(['species','latitude', 'longitude','wnvpresent','year','month','week']).sum().sort_values(by=['year', 'month','week', 'latitude' ,'longitude']).reset_index()

#to sum all the mosquitos in one as observed that every 50 mosquitos, will make extra row

In [9]:
train_df.head()

,species,latitude,longitude,wnvpresent,year,month,week,nummosquitos
0,CULEX PIPIENS/RESTUANS,41.688324,-87.676709,0,2007,5,22,1
1,CULEX RESTUANS,41.688324,-87.676709,0,2007,5,22,1
2,CULEX RESTUANS,41.720848,-87.666014,0,2007,5,22,3
3,CULEX PIPIENS,41.731922,-87.677512,0,2007,5,22,1
4,CULEX RESTUANS,41.731922,-87.677512,0,2007,5,22,5


In [10]:
train_df = pd.get_dummies(train_df, columns = ['species'],drop_first=True)
train_df
#get dummmie for species of mosquitos

,latitude,longitude,wnvpresent,year,month,week,nummosquitos,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS
0,41.688324,-87.676709,0,2007,5,22,1,0,1,0,0,0,0
1,41.688324,-87.676709,0,2007,5,22,1,0,0,1,0,0,0
2,41.720848,-87.666014,0,2007,5,22,3,0,0,1,0,0,0
3,41.731922,-87.677512,0,2007,5,22,1,1,0,0,0,0,0
4,41.731922,-87.677512,0,2007,5,22,5,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727,41.987280,-87.666066,0,2013,9,39,5,0,1,0,0,0,0
7728,41.991429,-87.747113,0,2013,9,39,2,0,1,0,0,0,0
7729,41.992478,-87.862995,0,2013,9,39,1,0,0,1,0,0,0
7730,42.008314,-87.777921,0,2013,9,39,10,0,1,0,0,0,0


In [11]:
train_df['species_UNSPECIFIED CULEX'] = 0
#this species is found in test but not in train. as data is not included, to add and put as 0

In [12]:
# latitude and longitude for station 1 and 2
stn1 = (41.995, -87.933)
stn2 = (41.786, -87.752)

In [13]:
def nearest_stn(lat, long):
    # set the lat, long coordinates for each point
    point = (lat, long)
    
    # calculate the dist between stn1 and point in km
    dist_1 = haversine(stn1, point)
    
    # calculate the dist between stn2 and point in km
    dist_2 = haversine(stn2, point)
    
    if dist_1 < dist_2:
        return 1
    else:
        return 2

In [14]:
# apply the mapping of nearest station to the mosquito counts data
train_df['station'] = train_df.apply(lambda x: nearest_stn(x['latitude'], x['longitude']), axis=1)

In [15]:
#feature engineer long lat
train_df['long_lat'] = train_df['longitude'] * train_df['latitude']

In [16]:
train_df.head()

,latitude,longitude,wnvpresent,year,month,week,nummosquitos,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX,station,long_lat
0,41.688324,-87.676709,0,2007,5,22,1,0,1,0,0,0,0,0,2,-3655.095052
1,41.688324,-87.676709,0,2007,5,22,1,0,0,1,0,0,0,0,2,-3655.095052
2,41.720848,-87.666014,0,2007,5,22,3,0,0,1,0,0,0,0,2,-3657.500445
3,41.731922,-87.677512,0,2007,5,22,1,1,0,0,0,0,0,0,2,-3658.951092
4,41.731922,-87.677512,0,2007,5,22,5,0,0,1,0,0,0,0,2,-3658.951092


In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7732 entries, 0 to 7731
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   latitude                        7732 non-null   float64
 1   longitude                       7732 non-null   float64
 2   wnvpresent                      7732 non-null   int64  
 3   year                            7732 non-null   int64  
 4   month                           7732 non-null   int64  
 5   week                            7732 non-null   int64  
 6   nummosquitos                    7732 non-null   int64  
 7   species_CULEX PIPIENS           7732 non-null   uint8  
 8   species_CULEX PIPIENS/RESTUANS  7732 non-null   uint8  
 9   species_CULEX RESTUANS          7732 non-null   uint8  
 10  species_CULEX SALINARIUS        7732 non-null   uint8  
 11  species_CULEX TARSALIS          7732 non-null   uint8  
 12  species_CULEX TERRITANS         77

In [18]:
#shift to wnvpresent and numoquitos to the front 
train_wnv = train_df['wnvpresent']
train_mos = train_df['nummosquitos']
train_df.drop(columns=['wnvpresent','nummosquitos'],inplace=True)

In [19]:
train_df.insert(0,'nummosquitos',train_wnv)
train_df.insert(1,'wnvpresent',train_wnv)

In [20]:
train_df.head()

,nummosquitos,wnvpresent,latitude,longitude,year,month,week,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX,station,long_lat
0,0,0,41.688324,-87.676709,2007,5,22,0,1,0,0,0,0,0,2,-3655.095052
1,0,0,41.688324,-87.676709,2007,5,22,0,0,1,0,0,0,0,2,-3655.095052
2,0,0,41.720848,-87.666014,2007,5,22,0,0,1,0,0,0,0,2,-3657.500445
3,0,0,41.731922,-87.677512,2007,5,22,1,0,0,0,0,0,0,2,-3658.951092
4,0,0,41.731922,-87.677512,2007,5,22,0,0,1,0,0,0,0,2,-3658.951092


In [21]:
train_df.duplicated(keep=False).sum()

0

 # Clean Weather

In [22]:
weather_wkly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tavg             434 non-null    float64
 1   dewpoint         434 non-null    float64
 2   wetbulb          434 non-null    float64
 3   heat             434 non-null    float64
 4   cool             434 non-null    float64
 5   preciptotal      434 non-null    float64
 6   stnpressure      434 non-null    float64
 7   resultdir        434 non-null    float64
 8   avgspeed         434 non-null    float64
 9   rel_humidity     434 non-null    float64
 10  year             434 non-null    int64  
 11  month            434 non-null    float64
 12  weekday          434 non-null    float64
 13  VCTS_week        434 non-null    int64  
 14  FG_week          434 non-null    int64  
 15  TSRA_week        434 non-null    int64  
 16  RA_week          434 non-null    int64  
 17  DZ_week         

In [23]:
#to reshift year week and month to the end
year = weather_wkly['year']
month= weather_wkly['month']
weekday = weather_wkly['weekday']

In [24]:
weather_wkly.drop(columns=['year','month','weekday'],inplace=True)

In [25]:
weather_wkly['year'] = year
weather_wkly['month']=month

In [26]:
weather_wkly.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   tavg             434 non-null    float64
 1   dewpoint         434 non-null    float64
 2   wetbulb          434 non-null    float64
 3   heat             434 non-null    float64
 4   cool             434 non-null    float64
 5   preciptotal      434 non-null    float64
 6   stnpressure      434 non-null    float64
 7   resultdir        434 non-null    float64
 8   avgspeed         434 non-null    float64
 9   rel_humidity     434 non-null    float64
 10  VCTS_week        434 non-null    int64  
 11  FG_week          434 non-null    int64  
 12  TSRA_week        434 non-null    int64  
 13  RA_week          434 non-null    int64  
 14  DZ_week          434 non-null    int64  
 15  BR_week          434 non-null    int64  
 16  HZ_week          434 non-null    int64  
 17  preciptotal_sum 

In [27]:
weather_shift = weather_wkly.iloc[:,0:18]
weather_shift_1 = []
for col in weather_shift.columns:
    weather_shift_1.append(col)
    print(col)
    
# this columns will be used to create extra columns to shift data for two and three weeks as we are predicting based onweeks

tavg
dewpoint
wetbulb
heat
cool
preciptotal
stnpressure
resultdir
avgspeed
rel_humidity
VCTS_week
FG_week
TSRA_week
RA_week
DZ_week
BR_week
HZ_week
preciptotal_sum


In [28]:
stn1_weather = weather_wkly[weather_wkly['station']==1] #separate to station 1
                                                        
stn1_weather.head()

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,TSRA_week,RA_week,DZ_week,BR_week,HZ_week,preciptotal_sum,week,station,year,month
0,58.500000,40.333333,49.333333,6.833333,0.333333,0.001667,29.358333,10.666667,12.050000,52.054022,...,0,1,0,1,0,0.010,18,1,2007,5.000000
2,63.428571,45.857143,54.571429,3.285714,1.714286,0.019286,29.341429,10.714286,8.642857,53.524463,...,0,1,0,3,1,0.135,19,1,2007,5.000000
4,61.571429,41.000000,50.571429,5.571429,2.142857,0.084286,29.355714,20.000000,11.814286,48.001934,...,0,3,0,2,0,0.590,20,1,2007,5.000000
6,67.142857,49.285714,57.571429,1.857143,4.000000,0.153571,29.382857,16.714286,10.428571,54.999156,...,1,3,0,2,2,1.075,21,1,2007,5.000000
8,72.142857,58.571429,63.857143,0.000000,7.142857,0.036429,29.207143,18.571429,7.542857,62.824094,...,2,4,0,4,5,0.255,22,1,2007,5.428571


In [29]:
stn2_weather = weather_wkly[weather_wkly['station']==2] #separate to station 2
                                                        
stn2_weather.head()

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,TSRA_week,RA_week,DZ_week,BR_week,HZ_week,preciptotal_sum,week,station,year,month
1,59.500000,40.666667,50.000000,6.000000,0.500000,0.000833,29.420000,9.500000,12.100000,50.793926,...,0,0,0,1,2,0.005,18,2,2007,5.000000
3,64.571429,45.857143,54.714286,3.000000,2.571429,0.003571,29.402857,12.714286,8.357143,51.630993,...,0,0,0,2,3,0.025,19,2,2007,5.000000
5,61.571429,40.571429,50.571429,5.428571,2.000000,0.111429,29.422857,18.714286,10.857143,47.283352,...,2,3,0,2,0,0.780,20,2,2007,5.000000
7,68.714286,49.285714,58.000000,1.428571,5.142857,0.065000,29.448571,15.428571,10.000000,52.336022,...,1,4,0,3,1,0.455,21,2,2007,5.000000
9,74.142857,58.714286,64.714286,0.000000,9.142857,0.152143,29.267143,17.142857,7.585714,59.072509,...,1,3,0,5,4,1.065,22,2,2007,5.428571


In [30]:
def shift_col(df, x): #function to shift data per period with the columns identified
    
    for i in weather_shift_1:
        name = i + str(x)
        df[name] = df[i].shift(x)
        
    return df

In [31]:
shift_col(stn1_weather,2) #create columns that for next two weeks in station1

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,avgspeed2,rel_humidity2,VCTS_week2,FG_week2,TSRA_week2,RA_week2,DZ_week2,BR_week2,HZ_week2,preciptotal_sum2
0,58.500000,40.333333,49.333333,6.833333,0.333333,0.001667,29.358333,10.666667,12.050000,52.054022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.428571,45.857143,54.571429,3.285714,1.714286,0.019286,29.341429,10.714286,8.642857,53.524463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61.571429,41.000000,50.571429,5.571429,2.142857,0.084286,29.355714,20.000000,11.814286,48.001934,...,12.050000,52.054022,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.010
6,67.142857,49.285714,57.571429,1.857143,4.000000,0.153571,29.382857,16.714286,10.428571,54.999156,...,8.642857,53.524463,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.135
8,72.142857,58.571429,63.857143,0.000000,7.142857,0.036429,29.207143,18.571429,7.542857,62.824094,...,11.814286,48.001934,1.0,0.0,0.0,3.0,0.0,2.0,0.0,0.590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,55.714286,46.571429,50.857143,10.142857,0.857143,0.155000,29.102857,20.714286,10.885714,71.602920,...,7.528571,65.364067,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.130
426,52.142857,37.857143,45.428571,12.857143,0.000000,0.003571,29.240000,19.571429,8.457143,58.632840,...,4.114286,60.835462,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.000
428,52.714286,48.142857,50.714286,12.285714,0.000000,0.194286,29.080000,23.857143,9.600000,84.580035,...,10.885714,71.602920,0.0,0.0,1.0,5.0,2.0,4.0,0.0,1.085
430,53.285714,39.857143,46.857143,11.714286,0.000000,0.001429,29.321429,16.571429,7.971429,61.339088,...,8.457143,58.632840,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.025


In [32]:
shift_col(stn1_weather,3) #create columns that for next 3 weeks in station1

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,58.500000,40.333333,49.333333,6.833333,0.333333,0.001667,29.358333,10.666667,12.050000,52.054022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.428571,45.857143,54.571429,3.285714,1.714286,0.019286,29.341429,10.714286,8.642857,53.524463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61.571429,41.000000,50.571429,5.571429,2.142857,0.084286,29.355714,20.000000,11.814286,48.001934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,67.142857,49.285714,57.571429,1.857143,4.000000,0.153571,29.382857,16.714286,10.428571,54.999156,...,12.050000,52.054022,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.010
8,72.142857,58.571429,63.857143,0.000000,7.142857,0.036429,29.207143,18.571429,7.542857,62.824094,...,8.642857,53.524463,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,55.714286,46.571429,50.857143,10.142857,0.857143,0.155000,29.102857,20.714286,10.885714,71.602920,...,9.271429,68.873221,0.0,0.0,0.0,3.0,0.0,2.0,0.0,1.450
426,52.142857,37.857143,45.428571,12.857143,0.000000,0.003571,29.240000,19.571429,8.457143,58.632840,...,7.528571,65.364067,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.130
428,52.714286,48.142857,50.714286,12.285714,0.000000,0.194286,29.080000,23.857143,9.600000,84.580035,...,4.114286,60.835462,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.000
430,53.285714,39.857143,46.857143,11.714286,0.000000,0.001429,29.321429,16.571429,7.971429,61.339088,...,10.885714,71.602920,0.0,0.0,1.0,5.0,2.0,4.0,0.0,1.085


In [33]:
shift_col(stn2_weather,2) #create columns that for next two weeks in station2

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,avgspeed2,rel_humidity2,VCTS_week2,FG_week2,TSRA_week2,RA_week2,DZ_week2,BR_week2,HZ_week2,preciptotal_sum2
1,59.500000,40.666667,50.000000,6.000000,0.500000,0.000833,29.420000,9.500000,12.100000,50.793926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64.571429,45.857143,54.714286,3.000000,2.571429,0.003571,29.402857,12.714286,8.357143,51.630993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,61.571429,40.571429,50.571429,5.428571,2.000000,0.111429,29.422857,18.714286,10.857143,47.283352,...,12.100000,50.793926,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.005
7,68.714286,49.285714,58.000000,1.428571,5.142857,0.065000,29.448571,15.428571,10.000000,52.336022,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
9,74.142857,58.714286,64.714286,0.000000,9.142857,0.152143,29.267143,17.142857,7.585714,59.072509,...,10.857143,47.283352,0.0,0.0,2.0,3.0,0.0,2.0,0.0,0.780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,58.571429,48.285714,53.142857,8.142857,1.714286,0.262857,29.168571,20.285714,10.857143,68.874099,...,8.071429,66.608892,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.130
427,55.285714,39.571429,47.428571,9.714286,0.000000,0.002857,29.302857,14.285714,8.500000,55.912086,...,4.957143,60.385470,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000
429,55.285714,50.428571,52.714286,9.714286,0.000000,0.246429,29.141429,24.285714,9.428571,83.763688,...,10.857143,68.874099,0.0,0.0,1.0,6.0,2.0,3.0,1.0,1.840
431,56.000000,41.142857,48.285714,9.000000,0.000000,0.000714,29.382857,22.714286,7.971429,58.214644,...,8.500000,55.912086,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.020


In [34]:
shift_col(stn2_weather,3) #create columns that for next three weeks in station2

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
1,59.500000,40.666667,50.000000,6.000000,0.500000,0.000833,29.420000,9.500000,12.100000,50.793926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64.571429,45.857143,54.714286,3.000000,2.571429,0.003571,29.402857,12.714286,8.357143,51.630993,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,61.571429,40.571429,50.571429,5.428571,2.000000,0.111429,29.422857,18.714286,10.857143,47.283352,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,68.714286,49.285714,58.000000,1.428571,5.142857,0.065000,29.448571,15.428571,10.000000,52.336022,...,12.100000,50.793926,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.005
9,74.142857,58.714286,64.714286,0.000000,9.142857,0.152143,29.267143,17.142857,7.585714,59.072509,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,58.571429,48.285714,53.142857,8.142857,1.714286,0.262857,29.168571,20.285714,10.857143,68.874099,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.730
427,55.285714,39.571429,47.428571,9.714286,0.000000,0.002857,29.302857,14.285714,8.500000,55.912086,...,8.071429,66.608892,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.130
429,55.285714,50.428571,52.714286,9.714286,0.000000,0.246429,29.141429,24.285714,9.428571,83.763688,...,4.957143,60.385470,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000
431,56.000000,41.142857,48.285714,9.000000,0.000000,0.000714,29.382857,22.714286,7.971429,58.214644,...,10.857143,68.874099,0.0,0.0,1.0,6.0,2.0,3.0,1.0,1.840


In [35]:
weather_recombined = pd.concat([stn1_weather, stn2_weather], axis=0)
weather_recombined #recombine station1 and station2

,tavg,dewpoint,wetbulb,heat,cool,preciptotal,stnpressure,resultdir,avgspeed,rel_humidity,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,58.500000,40.333333,49.333333,6.833333,0.333333,0.001667,29.358333,10.666667,12.050000,52.054022,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.428571,45.857143,54.571429,3.285714,1.714286,0.019286,29.341429,10.714286,8.642857,53.524463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,61.571429,41.000000,50.571429,5.571429,2.142857,0.084286,29.355714,20.000000,11.814286,48.001934,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,67.142857,49.285714,57.571429,1.857143,4.000000,0.153571,29.382857,16.714286,10.428571,54.999156,...,12.050000,52.054022,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.010
8,72.142857,58.571429,63.857143,0.000000,7.142857,0.036429,29.207143,18.571429,7.542857,62.824094,...,8.642857,53.524463,0.0,0.0,0.0,1.0,0.0,3.0,1.0,0.135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,58.571429,48.285714,53.142857,8.142857,1.714286,0.262857,29.168571,20.285714,10.857143,68.874099,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.730
427,55.285714,39.571429,47.428571,9.714286,0.000000,0.002857,29.302857,14.285714,8.500000,55.912086,...,8.071429,66.608892,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.130
429,55.285714,50.428571,52.714286,9.714286,0.000000,0.246429,29.141429,24.285714,9.428571,83.763688,...,4.957143,60.385470,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000
431,56.000000,41.142857,48.285714,9.000000,0.000000,0.000714,29.382857,22.714286,7.971429,58.214644,...,10.857143,68.874099,0.0,0.0,1.0,6.0,2.0,3.0,1.0,1.840


In [36]:
weather_recombined.drop(columns=['month'],axis=1,inplace=True) 
#drop weekday and month as it is not useful for the weather

In [37]:
weather_recombined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 434 entries, 0 to 433
Data columns (total 57 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tavg              434 non-null    float64
 1   dewpoint          434 non-null    float64
 2   wetbulb           434 non-null    float64
 3   heat              434 non-null    float64
 4   cool              434 non-null    float64
 5   preciptotal       434 non-null    float64
 6   stnpressure       434 non-null    float64
 7   resultdir         434 non-null    float64
 8   avgspeed          434 non-null    float64
 9   rel_humidity      434 non-null    float64
 10  VCTS_week         434 non-null    int64  
 11  FG_week           434 non-null    int64  
 12  TSRA_week         434 non-null    int64  
 13  RA_week           434 non-null    int64  
 14  DZ_week           434 non-null    int64  
 15  BR_week           434 non-null    int64  
 16  HZ_week           434 non-null    int64  
 1

# Combine Train and Weather

In [38]:
combined = train_df.merge(weather_recombined, how='left', on=['year','week','station'])
combined #merge with train with year,week and station

,nummosquitos,wnvpresent,latitude,longitude,year,month,week,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,0,0,41.688324,-87.676709,2007,5,22,0,1,0,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
1,0,0,41.688324,-87.676709,2007,5,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
2,0,0,41.720848,-87.666014,2007,5,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
3,0,0,41.731922,-87.677512,2007,5,22,1,0,0,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
4,0,0,41.731922,-87.677512,2007,5,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7727,0,0,41.987280,-87.666066,2013,9,39,0,1,0,...,8.000000,65.110218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015
7728,0,0,41.991429,-87.747113,2013,9,39,0,1,0,...,8.000000,65.110218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015
7729,0,0,41.992478,-87.862995,2013,9,39,0,0,1,...,8.000000,65.110218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015
7730,0,0,42.008314,-87.777921,2013,9,39,0,1,0,...,8.000000,65.110218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015


In [39]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7732 entries, 0 to 7731
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   nummosquitos                    7732 non-null   int64  
 1   wnvpresent                      7732 non-null   int64  
 2   latitude                        7732 non-null   float64
 3   longitude                       7732 non-null   float64
 4   year                            7732 non-null   int64  
 5   month                           7732 non-null   int64  
 6   week                            7732 non-null   int64  
 7   species_CULEX PIPIENS           7732 non-null   uint8  
 8   species_CULEX PIPIENS/RESTUANS  7732 non-null   uint8  
 9   species_CULEX RESTUANS          7732 non-null   uint8  
 10  species_CULEX SALINARIUS        7732 non-null   uint8  
 11  species_CULEX TARSALIS          7732 non-null   uint8  
 12  species_CULEX TERRITANS         77

In [40]:
#reshift to latitue and longitude columns
test_lat = combined['latitude']
test_long = combined['longitude']

In [41]:
combined.drop(columns=['latitude','longitude'],axis=1,inplace=True)
combined.insert(4, 'latitude', test_lat)
combined.insert(5, 'longitude', test_long)

In [42]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7732 entries, 0 to 7731
Data columns (total 70 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   nummosquitos                    7732 non-null   int64  
 1   wnvpresent                      7732 non-null   int64  
 2   year                            7732 non-null   int64  
 3   month                           7732 non-null   int64  
 4   latitude                        7732 non-null   float64
 5   longitude                       7732 non-null   float64
 6   week                            7732 non-null   int64  
 7   species_CULEX PIPIENS           7732 non-null   uint8  
 8   species_CULEX PIPIENS/RESTUANS  7732 non-null   uint8  
 9   species_CULEX RESTUANS          7732 non-null   uint8  
 10  species_CULEX SALINARIUS        7732 non-null   uint8  
 11  species_CULEX TARSALIS          7732 non-null   uint8  
 12  species_CULEX TERRITANS         77

In [43]:
combined.duplicated(keep=False).sum() #check duplication

0

In [44]:
combined.head()

,nummosquitos,wnvpresent,year,month,latitude,longitude,week,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,0,0,2007,5,41.688324,-87.676709,22,0,1,0,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
1,0,0,2007,5,41.688324,-87.676709,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
2,0,0,2007,5,41.720848,-87.666014,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
3,0,0,2007,5,41.731922,-87.677512,22,1,0,0,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025
4,0,0,2007,5,41.731922,-87.677512,22,0,0,1,...,8.357143,51.630993,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.025


In [45]:
combined.to_csv('../assets/combined1.csv', index=False) #save the csv

# Clean Test

In [46]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 11 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      116293 non-null  int64  
 1   date                    116293 non-null  object 
 2   address                 116293 non-null  object 
 3   species                 116293 non-null  object 
 4   block                   116293 non-null  int64  
 5   street                  116293 non-null  object 
 6   trap                    116293 non-null  object 
 7   addressnumberandstreet  116293 non-null  object 
 8   latitude                116293 non-null  float64
 9   longitude               116293 non-null  float64
 10  addressaccuracy         116293 non-null  int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 9.8+ MB


In [47]:
test_df['date'] = pd.to_datetime(test_df['date'])  #convert to datetime and create new columns for year month and week
test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['week'] = test_df['date'].dt.isocalendar().week

In [48]:
test_df.drop(['date'],inplace=True,axis=1) #drop date as it wont be useful for the model any longer

In [49]:
test_df = pd.get_dummies(test_df, columns = ['species'],drop_first=True)
test_df

,id,address,block,street,trap,addressnumberandstreet,latitude,longitude,addressaccuracy,year,month,week,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,species_CULEX TERRITANS,species_UNSPECIFIED CULEX
0,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2008,6,24,0,1,0,0,0,0,0
1,2,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2008,6,24,0,0,1,0,0,0,0
2,3,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2008,6,24,1,0,0,0,0,0,0
3,4,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2008,6,24,0,0,0,1,0,0,0
4,5,"4100 North Oak Park Avenue, Chicago, IL 60634,...",41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,2008,6,24,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116288,116289,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.633590,8,2014,10,40,0,0,0,1,0,0,0
116289,116290,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.633590,8,2014,10,40,0,0,0,0,0,1,0
116290,116291,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.633590,8,2014,10,40,0,0,0,0,1,0,0
116291,116292,"2100 North Cannon Drive, Chicago, IL 60614, USA",21,N CANNON DR,T054C,"2100 N CANNON DR, Chicago, IL",41.925652,-87.633590,8,2014,10,40,0,0,0,0,0,0,1


In [50]:
test_df['station'] = test_df.apply(lambda x: nearest_stn(x['latitude'], x['longitude']), axis=1) 
#create new columns station based on latitude and longitude

In [51]:
test_df['long_lat'] = test_df['longitude'] * test_df['latitude']

In [52]:
test_df.drop(columns =
             ['id', 'address','block', 'street','addressnumberandstreet', 'addressaccuracy','trap'],
             axis=1, 
          inplace= True)
#drop this columns as not necessary for the model

In [53]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   latitude                        116293 non-null  float64
 1   longitude                       116293 non-null  float64
 2   year                            116293 non-null  int64  
 3   month                           116293 non-null  int64  
 4   week                            116293 non-null  UInt32 
 5   species_CULEX PIPIENS           116293 non-null  uint8  
 6   species_CULEX PIPIENS/RESTUANS  116293 non-null  uint8  
 7   species_CULEX RESTUANS          116293 non-null  uint8  
 8   species_CULEX SALINARIUS        116293 non-null  uint8  
 9   species_CULEX TARSALIS          116293 non-null  uint8  
 10  species_CULEX TERRITANS         116293 non-null  uint8  
 11  species_UNSPECIFIED CULEX       116293 non-null  uint8  
 12  station         

In [54]:
#shift longitued and langitude
test_lat = test_df['latitude']
test_long = test_df['longitude']

In [55]:
test_df.drop(columns=['latitude','longitude'],axis=1,inplace=True)
test_df.insert(3, 'latitude', test_lat)
test_df.insert(4, 'longitude', test_long)

In [56]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116293 entries, 0 to 116292
Data columns (total 14 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   year                            116293 non-null  int64  
 1   month                           116293 non-null  int64  
 2   week                            116293 non-null  UInt32 
 3   latitude                        116293 non-null  float64
 4   longitude                       116293 non-null  float64
 5   species_CULEX PIPIENS           116293 non-null  uint8  
 6   species_CULEX PIPIENS/RESTUANS  116293 non-null  uint8  
 7   species_CULEX RESTUANS          116293 non-null  uint8  
 8   species_CULEX SALINARIUS        116293 non-null  uint8  
 9   species_CULEX TARSALIS          116293 non-null  uint8  
 10  species_CULEX TERRITANS         116293 non-null  uint8  
 11  species_UNSPECIFIED CULEX       116293 non-null  uint8  
 12  station         

# Combine test and weather

In [57]:
combined_t = test_df.merge(weather_recombined, how='left', on=['year','week','station'])
combined_t

,year,month,week,latitude,longitude,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,2008,6,24,41.954690,-87.800991,0,1,0,0,0,...,9.300000,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
1,2008,6,24,41.954690,-87.800991,0,0,1,0,0,...,9.300000,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
2,2008,6,24,41.954690,-87.800991,1,0,0,0,0,...,9.300000,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
3,2008,6,24,41.954690,-87.800991,0,0,0,1,0,...,9.300000,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
4,2008,6,24,41.954690,-87.800991,0,0,0,0,0,...,9.300000,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116288,2014,10,40,41.925652,-87.633590,0,0,0,1,0,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.73
116289,2014,10,40,41.925652,-87.633590,0,0,0,0,0,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.73
116290,2014,10,40,41.925652,-87.633590,0,0,0,0,1,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.73
116291,2014,10,40,41.925652,-87.633590,0,0,0,0,0,...,9.214286,70.318189,0.0,0.0,0.0,3.0,2.0,2.0,0.0,1.73


In [58]:
combined_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116293 entries, 0 to 116292
Data columns (total 68 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   year                            116293 non-null  int64  
 1   month                           116293 non-null  int64  
 2   week                            116293 non-null  UInt32 
 3   latitude                        116293 non-null  float64
 4   longitude                       116293 non-null  float64
 5   species_CULEX PIPIENS           116293 non-null  uint8  
 6   species_CULEX PIPIENS/RESTUANS  116293 non-null  uint8  
 7   species_CULEX RESTUANS          116293 non-null  uint8  
 8   species_CULEX SALINARIUS        116293 non-null  uint8  
 9   species_CULEX TARSALIS          116293 non-null  uint8  
 10  species_CULEX TERRITANS         116293 non-null  uint8  
 11  species_UNSPECIFIED CULEX       116293 non-null  uint8  
 12  station         

In [59]:
combined_t.head()

,year,month,week,latitude,longitude,species_CULEX PIPIENS,species_CULEX PIPIENS/RESTUANS,species_CULEX RESTUANS,species_CULEX SALINARIUS,species_CULEX TARSALIS,...,avgspeed3,rel_humidity3,VCTS_week3,FG_week3,TSRA_week3,RA_week3,DZ_week3,BR_week3,HZ_week3,preciptotal_sum3
0,2008,6,24,41.95469,-87.800991,0,1,0,0,0,...,9.3,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
1,2008,6,24,41.95469,-87.800991,0,0,1,0,0,...,9.3,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
2,2008,6,24,41.95469,-87.800991,1,0,0,0,0,...,9.3,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
3,2008,6,24,41.95469,-87.800991,0,0,0,1,0,...,9.3,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01
4,2008,6,24,41.95469,-87.800991,0,0,0,0,0,...,9.3,56.409332,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.01


In [60]:
combined_t.to_csv('../assets/test1.csv', index=False)